# 概率

- 联合概率、条件概率与相互独立
    - 联合概率：包含多个条件，且所有条件同时成立的概率:记作：P(A,B)
    - 条件概率：就是事件A在另外一个事件B已经发生条件下的发生概率,记作：P(A|B)
    - 相互独立：如果P(A, B) = P(A)P(B)，则称事件A与事件B相互独立。

贝叶斯公式

![avatar](../source/137.jpg) 
![avatar](../source/138.jpg) 
![avatar](../source/139.jpg) 
![avatar](../source/140.jpg) 

# 案例：商品评论情感分析
- sklearn.naive_bayes.MultinomialNB(alpha = 1.0)
    - 朴素贝叶斯分类
    - alpha：拉普拉斯平滑系数
![avatar](../source/141.jpg)


## 步骤分析
- 1）获取数据
- 2）数据基本处理
    - 2.1） 取出内容列，对数据进行分析
    - 2.2） 判定评判标准
    - 2.3） 选择停用词
    - 2.4） 把内容处理，转化成标准格式
    - 2.5） 统计词的个数
    - 2.6）准备训练集和测试集
- 3）模型训练
- 4）模型评估

In [96]:
import pandas as pd
import numpy as np
import jieba
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

## 获取数据

In [122]:
# 加载数据
data = pd.read_csv("../data/书籍评价.csv", encoding="gbk")
data.columns = ['index', 'content', 'eval']
data.head()

,index,content,eval
0,0,从编程小白的角度看，入门极佳。,好评
1,1,很好的入门书，简洁全面，适合小白。,好评
2,2,讲解全面，许多小细节都有顾及，三个小项目受益匪浅。,好评
3,3,前半部分讲概念深入浅出，要言不烦，很赞,好评
4,4,看了一遍还是不会写，有个概念而已,差评


In [108]:
# 2.1） 取出内容列，对数据进行分析
content = data["content"]
content


0                从编程小白的角度看，入门极佳。
1              很好的入门书，简洁全面，适合小白。
2      讲解全面，许多小细节都有顾及，三个小项目受益匪浅。
3            前半部分讲概念深入浅出，要言不烦，很赞
4               看了一遍还是不会写，有个概念而已
5           中规中矩的教科书，零基础的看了依旧看不懂
6       内容太浅显，个人认为不适合有其它语言编程基础的人
7                           破书一本
8     适合完完全全的小白读，有其他语言经验的可以去看别的书
9                     基础知识写的挺好的！
10                           太基础
11            略_嗦。。适合完全没有编程经验的小白
12                      真的真的不建议买
Name: content, dtype: object

In [109]:
data.loc[data.loc[:, 'eval'] == '好评', 'score']=1
data.loc[data.loc[:, 'eval'] == '差评', 'score']=0
data

,index,content,eval,score
0,0,从编程小白的角度看，入门极佳。,好评,1.0
1,1,很好的入门书，简洁全面，适合小白。,好评,1.0
2,2,讲解全面，许多小细节都有顾及，三个小项目受益匪浅。,好评,1.0
3,3,前半部分讲概念深入浅出，要言不烦，很赞,好评,1.0
4,4,看了一遍还是不会写，有个概念而已,差评,0.0
5,5,中规中矩的教科书，零基础的看了依旧看不懂,差评,0.0
6,6,内容太浅显，个人认为不适合有其它语言编程基础的人,差评,0.0
7,7,破书一本,差评,0.0
8,8,适合完完全全的小白读，有其他语言经验的可以去看别的书,差评,0.0
9,9,基础知识写的挺好的！,好评,1.0


In [110]:
stopwords=[]
with open('../data/stopwords.txt','r',encoding='utf-8') as f:
    lines=f.readlines()
    for tmp in lines:
        line=tmp.strip()
        stopwords.append(line)
stopwords=list(set(stopwords))#去重  列表形式
stopwords[:5]

['', '咚', '限制', '从宽', '传说']

In [111]:
comment_list = []
for tmp in content:
    # cut_all 参数默认为 False,所有使用 cut 方法时默认为精确模式
    seg_list = jieba.cut(tmp, cut_all=False)
    #print(seg_list)  # <generator object Tokenizer.cut at 0x0000000007CF7DB0>
    seg_str = ','.join(seg_list)  # 拼接字符串
    comment_list.append(seg_str)  # 目的是转化成列表形式
comment_list

[' ,从,编程,小白,的,角度看,，,入门,极佳,。',
 '很,好,的,入门,书,，,简洁,全面,，,适合,小白,。',
 '讲解,全面,，,许多,小,细节,都,有,顾及,，,三个,小,项目,受益匪浅,。',
 '前半部,分讲,概念,深入浅出,，,要言不烦,，,很赞',
 '看,了,一遍,还是,不会,写,，,有个,概念,而已',
 '中规中矩,的,教科书,，,零,基础,的,看,了,依旧,看不懂',
 '内容,太,浅显,，,个人,认为,不,适合,有,其它,语言,编程,基础,的,人',
 '破书,一本',
 '适合,完完全全,的,小白读,，,有,其他,语言,经验,的,可以,去,看,别的,书',
 '基础知识,写,的,挺,好,的,！',
 '太,基础',
 '略,_,嗦,。,。,适合,完全,没有,编程,经验,的,小白',
 '真的,真的,不,建议,买']

In [117]:
# 2.5） 统计词的个数
# CountVectorizer 类会将文本中的词语转换为词频矩阵
con = CountVectorizer(stop_words=stopwords)
# 进行词数统计
X = con.fit_transform(comment_list)  # 它通过 fit_transform 函数计算各个词语出现的次数
name = con.get_feature_names()  # 通过 get_feature_names()可获取词袋中所有文本的关键字
print(X.toarray())  # 通过 toarray()可看到词频矩阵的结果
print(name)

[[0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0
  0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0
  0]
 [0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1
  1]
 [0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0
  0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0]
 [0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
  0]
 [0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 1 0
  0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
  0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0
  0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0
  0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 2 

<13x37 sparse matrix of type '<class 'numpy.int64'>'
	with 50 stored elements in Compressed Sparse Row format>

In [113]:
good_or_bad = data['score'].values  # 获取数据
good_or_bad

array([1., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.])

In [118]:
x_train = X.toarray()[:10, :]
y_train = good_or_bad[:10]
# 准备测试集
x_test = X.toarray()[10:, :]
y_test = good_or_bad[10:]

In [119]:
# 构建贝叶斯算法分类器
mb = MultinomialNB(alpha=1)  # alpha 为可选项，默认 1.0，添加拉普拉修/Lidstone 平滑参数
# 训练数据
mb.fit(x_train, y_train)
# 预测数据
y_predict = mb.predict(x_test)
#预测值与真实值展示
print('预测值：',y_predict)
print('真实值：',y_test)

预测值： [0. 0. 0.]
真实值： [0. 0. 0.]


In [120]:
mb.score(x_text, y_text)

1.0

百度情感分析:https://ai.baidu.com/tech/nlp_apply/sentiment_classify

## 朴素贝叶斯算法总结

- 优点：
    - 朴素贝叶斯模型发源于古典数学理论，有稳定的分类效率
    - 对缺失数据不太敏感，算法也比较简单，常用于文本分类
    - 分类准确度高，速度快
    
    
- 缺点：
    - 由于使用了样本属性独立性的假设，所以如果特征属性有关联时其效果不好
    - 需要计算先验概率，而先验概率很多时候取决于假设，假设的模型可以有很多种，因此在某些时候会由于假设的先验模型的原因导致预测效果不佳；
    
    
- NB的原理
    - 朴素贝叶斯法是基于贝叶斯定理与特征条件独立假设的分类方法。
    - 对于给定的待分类项x，通过学习到的模型计算后验概率分布，
    - 即：在此项出现的条件下各个目标类别出现的概率，将后验概率最大的类作为xx所属的类别。


- 朴素贝叶斯朴素在哪里？
    - 在计算条件概率分布P(X=x∣Y=c_k)时，NB引入了一个很强的条件独立假设，即，当Y确定时，X的各个特征分量取值之间相互独立。
    
    
- 为什么引入条件独立性假设？
    - 为了避免贝叶斯定理求解时面临的组合爆炸、样本稀疏问题。
    
    
- 在估计条件概率P(X∣Y)时出现概率为0的情况怎么办？
    - 这个情况通常见于数据量不足,解决这一问题的方法是采用贝叶斯估计。
    - 简单来说，引入λ，
        - 当λ=0时，就是普通的极大似然估计；
        - 当λ=1时称为拉普拉斯平滑
   
   
- 为什么属性独立性假设在实际情况中很难成立，但朴素贝叶斯仍能取得较好的效果?
    - 人们在使用分类器之前，首先做的第一步（也是最重要的一步）往往是特征选择，这个过程的目的就是为了排除特征之间的共线性、选择相对较为独立的特征；
    - 对于分类任务来说，只要各类别的条件概率排序正确，无需精准概率值就可以得出正确分类；
    - 如果属性间依赖对所有类别影响相同，或依赖关系的影响能相互抵消，则属性条件独立性假设在降低计算复杂度的同时不会对性能产生负面影响
    
    
- 朴素贝叶斯与LR(逻辑回归)的区别？
    - 朴素贝叶斯是生成模型, 而LR是判别模型
    - 朴素贝叶斯是基于很强的条件独立假设（在已知分类Y的条件下，各个特征变量取值是相互独立的），而LR则对此没有要求；
    - 朴素贝叶斯适用于数据集少的情景, 而LR适用于大规模数据集